In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from sympy import *

In [ ]:
data = {}
subdir = "../data/LX182,LY182"
for _f in os.listdir(subdir):
    if _f.endswith(".000.dat"):
        # split the time which comes after tmp_nAB_f_t and before the first dot
        _t = _f.split("tmp_nAB_f_t")[1].split(".")[0]
        print(_f, _t)
        # read the data, adding to data dictionary
        data[_t] = np.loadtxt(os.path.join(subdir, _f), skiprows=1)

# sort data by ascending time converted to int
data = dict(sorted(data.items(), key=lambda x: int(x[0])))


In [ ]:
# get list of keys from data dictionary
keys = list(data.keys())
times = np.array(list(data.keys()), dtype=float)
times

In [ ]:
# Statics
_ix = data["0"][:,0].astype(int)
_iy = data["0"][:,1].astype(int)
_dx = data["0"][:,2]
_dy = data["0"][:,3]

Lx = _ix.max()
Ly = _iy.max()
print(Lx, Ly)

iX, iY = _ix.reshape(Lx, Ly), _iy.reshape(Lx, Ly)
dX, dY = _dx.reshape(Lx, Ly), _dy.reshape(Lx, Ly)

In [ ]:
data[keys[0]][:,4].reshape(Lx, Ly)[0,:]

In [ ]:
# previous plot using plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=iY[0,:], y=data[keys[0]][:,4].reshape(Lx, Ly).mean(axis=0)))
fig.add_trace(go.Scatter(x=[0], y=[0]))
fig.show()

In [ ]:
# data
_data = data[keys[-1]][:,4].reshape(Lx, Ly)
_xMean = data[keys[-1]][:,4].reshape(Lx, Ly).mean(axis=0)
_yMean = data[keys[-1]][:,4].reshape(Lx, Ly).mean(axis=1)

_xMeanMax = _xMean.max()
_xMeanMin = _xMean.min()
_xMeanDelta = np.abs(_xMeanMax - _xMeanMin)

_yMeanMax = _yMean.max()
_yMeanMin = _yMean.min()
_yMeanDelta = np.abs(_yMeanMax - _yMeanMin)

# plotly subplot with trace on left and heatmap on right
fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.1, 0.9], row_heights=[0.9, 0.1],
    vertical_spacing=0.02, horizontal_spacing=0.02,
    subplot_titles=('', f'Relaxed {Lx}x{Ly}, t={keys[-1]}', ''),
    shared_xaxes = True, shared_yaxes=True  # Remove shared_xaxes
)

fig.add_trace(go.Scatter(x=_xMean, y=iY[0,:]), row=1, col=1)
fig.add_trace(go.Heatmap(z=_data.T, showscale=False, colorscale='Plasma'), row=1, col=2)
fig.add_trace(go.Scatter(x=iX[:,0], y=_yMean), row=2, col=2)

fig.add_shape(type="rect", x0=0, y0=100, x1=Lx-1, y1=105, line=dict(color="RoyalBlue"), fillcolor='LightSkyBlue', opacity=0.5, row=1, col=2)
fig.add_shape(type="rect", x0=100, y0=0, x1=105, y1=Ly-1, line=dict(color="Teal"), fillcolor='LightSeaGreen', opacity=0.5, row=1, col=2)

# Update layout for scatter plot (col1)
fig.update_xaxes(title_text='density',
                 range=[_xMeanMin - _xMeanDelta*0.1, _xMeanMax + _xMeanDelta*0.1],
                 row=1, col=1, side='top', showticklabels=True)  # Add side='top' to move x-axis to the top
fig.update_yaxes(title_text='', range=[0, Ly], row=1, col=1)

# Update layout for heatmap (col2)
fig.update_xaxes(scaleanchor='y2', range=[0, Lx], constrain='domain', scaleratio=1, row=1, col=2, showticklabels=False)
fig.update_yaxes(scaleanchor='x2', range=[0, Ly], constrain='domain', scaleratio=1, row=1, col=2, showticklabels=False)

# Update layout for scatter plot (row 2, col 2)
fig.update_yaxes(title_text='density', range=[_yMeanMin - _yMeanDelta*0.1, _yMeanMax + _yMeanDelta*0.1], row=2, col=2, side='right', showticklabels=True)

# Update overall layout
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=40, b=20)
)

fig.show()

In [ ]:
# identify the peaks of _yMean
_yMeanPeaks = peak_local_max(_yMean, min_distance=3).flatten()

# sort the peaks by index
_yMeanPeaks = _yMeanPeaks[np.argsort(_yMeanPeaks)]

# plot the peaks
fig = go.Figure()
fig.add_trace(go.Scatter(x=iX[:,0], y=_yMean))
fig.add_trace(go.Scatter(x=iX[_yMeanPeaks,0], y=_yMean[_yMeanPeaks], mode='markers', marker=dict(size=10, color='red')))
fig.show()


In [ ]:
# identify the peaks of _xMean
_xMeanPeaks = peak_local_max(_xMean, min_distance=3).flatten()

# sort the peaks by index
_xMeanPeaks = _xMeanPeaks[np.argsort(_xMeanPeaks)]

# plot the peaks
fig = go.Figure()
fig.add_trace(go.Scatter(x=iY[0,:], y=_xMean))
fig.add_trace(go.Scatter(x=iY[0,_xMeanPeaks], y=_xMean[_xMeanPeaks], mode='markers', marker=dict(size=10, color='red')))
fig.show()


In [ ]:
t_iX[:,0]

In [ ]:
# Test data set
t_pA = data[keys[-1]][:,4].reshape(Lx, Ly)[8:172, 4:176]
t_iX = iX[8:172, 4:176]
t_iY = iY[8:172, 4:176]

# plot the test data set
fig = go.Figure()
fig.add_trace(go.Heatmap(x=t_iX[:,0], y=t_iY[0,:], z=t_pA.T, showscale=False, colorscale='Plasma'))

# Update overall layout to keep aspect ratio
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=40, b=20),
    xaxis=dict(
        constrain='domain',
        scaleanchor="y",
        scaleratio=1
    ),
    yaxis=dict(
        constrain='domain',
        scaleanchor="x",
        scaleratio=1
    )
)

fig.show()


In [ ]:
pA = data[keys[-1]][:,4].reshape(Lx, Ly)
pA.mean(), t_pA.mean()

In [ ]:
_radii = np.linspace(1, Lx*0.8, 100)
_rMean = np.zeros_like(_radii)
for _i, _r in enumerate(_radii):
    _rMean[_i] = pA[(iX-Lx//2)**2 + (iY-Ly//2)**2 < _r**2].mean()

plt.plot(_radii, _rMean)
print(_rMean[-5:].mean(), _rMean[-5:].std())

In [ ]:
_t_radii = np.linspace(1, Lx*0.8, 100)
_t_rMean = np.zeros_like(_radii)
for _i, _r in enumerate(_radii):
    _rMean[_i] = t_pA[(t_iX-Lx//2)**2 + (t_iY-Ly//2)**2 < _r**2].mean()

plt.plot(_radii, _rMean)
print(_rMean[-5:].mean(), _rMean[-5:].std())

In [ ]:
# create a gaussian blur of t_pA data
from scipy.ndimage import gaussian_filter
t_pA_blur = gaussian_filter(t_pA, sigma=10.5)

# plot the test data set
fig = go.Figure()
fig.add_trace(go.Heatmap(x=t_iX[:,0], y=t_iY[0,:], z=t_pA_blur.T, showscale=False, colorscale='Plasma'))

# Update overall layout to keep aspect ratio
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=40, b=20),
    xaxis=dict(
        constrain='domain',
        scaleanchor="y",
        scaleratio=1
    ),
    yaxis=dict(
        constrain='domain',
        scaleanchor="x",
        scaleratio=1
    )
)

In [ ]:
# report value from center of t_pA_blur
t_pA_blur[t_pA.shape[0]//2, t_pA.shape[1]//2], t_pA_blur.mean(), t_pA_blur.std()

In [ ]:
t_pA.shape

In [ ]:
_t_sigma = np.linspace(6, 30, 600)
_t_pA_blur_center = np.zeros_like(_t_sigma)
for _i, _sigma in enumerate(_t_sigma):
    # _t_pA_blur_center[_i] = gaussian_filter(t_pA, sigma=_sigma)[t_pA.shape[0]//2, t_pA.shape[1]//2]
    _t_pA_blur_center[_i] = gaussian_filter(t_pA, sigma=_sigma)[t_pA.shape[0]//2, t_pA.shape[1]//2]

fig = go.Figure()
fig.add_trace(go.Scatter(y=_t_pA_blur_center))
fig.show()
